In [1]:
import urllib.request
import os
import zipfile

url = "https://www.insee.fr/fr/statistiques/fichier/4648335/bdf2017fe_csv.zip"
filename = "bdf2017fe_csv.zip"

# Define the destination directory
dest_dir = 'already_downloaded_data'

# Create the destination directory if it doesn't exist
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

# Join the destination directory with the filename to create the full destination path
dest_path = os.path.join(dest_dir, filename)

# Download the file and save it to the destination path
urllib.request.urlretrieve(url, dest_path)

# Extract the contents of the zip file to the destination directory
with zipfile.ZipFile(dest_path, 'r') as zip_ref:
    zip_ref.extractall(dest_dir)

In [2]:
import pandas as pd

csv_filename = "tf106.csv"
csv_filepath = os.path.join(dest_dir, csv_filename)

df = pd.read_csv(csv_filepath, sep=";")

In [3]:
# filtering values with exactly 3 characters, corresponding to ECOICOP groups
df = df.loc[df['NOMENCLATURE'].str.len() == 3]

In [4]:
# filtering out rows where the "FCOIC" value starts with "13"
df = df.loc[~df['NOMENCLATURE'].str.startswith('13')]

Now let's deal with Eurostat inflation stats.

In [7]:
import pandas as pd

# this file is around 200 mbs so it can take some time to download

df_inflation = pd.read_csv("https://ec.europa.eu/eurostat/databrowser-backend/api/extraction/1.0/LIVE/true/sdmx/csv/PRC_HICP_MANR?i")


In [8]:
df_inflation = df_inflation[df_inflation["geo"] == "FR"]

In [9]:
# filter the rows using a regular expression, keeping only the ECOICOP groups (3 numbers)
df_inflation = df_inflation[df_inflation['coicop'].str.match(r'^CP\d{3}$')]

In [10]:
# remove the "CP" prefix from the values in the coicop column
df_inflation['coicop'] = df_inflation['coicop'].str.slice(start=2)

In [11]:
df_inflation = df_inflation[df_inflation['TIME_PERIOD'].isin(['2022-03', '2023-03'])]

In [12]:
# keep only the specified columns using the .loc[] indexer
df_inflation = df_inflation.loc[:, ['coicop', 'TIME_PERIOD', 'OBS_VALUE']]

In [13]:
# pivot the dataframe to collapse duplicate values in the coicop column
df_inflation = df_inflation.pivot(index='coicop', columns='TIME_PERIOD', values='OBS_VALUE')


In [14]:
# calculate the Cumulated inflation column based on the values in 2022-03 and 2023-03
df_inflation['Cumulated'] = ((1 + (df_inflation['2022-03'] / 100)) * (1 + (df_inflation['2023-03'] / 100))-1)*100

A REPARER : Il y a 40 valeurs de coicop ici, alors que y en a 47 dans l'autre tableau, à vérifier où sont les 7 manquantes


In [22]:
merged = pd.merge(df, df_inflation, left_on='NOMENCLATURE', right_index=True)


In [23]:
# calculating the sum of "CONSO" for each unique value in "DECUC" column
sum_by_decuc = merged.groupby('DECUC')['CONSO'].sum()

In [24]:
# adding a new column "PROP" with the percentage of each row relative to the sum for each unique value in "DECUC" column
merged['PROP'] = merged['CONSO'] / merged['DECUC'].map(sum_by_decuc)

Let's make sure sumPROP is equal to 1

In [27]:
sumPROP = merged.groupby('DECUC')['PROP'].sum()
sumPROP

DECUC
1      1.0
10     1.0
2      1.0
3      1.0
4      1.0
5      1.0
6      1.0
7      1.0
8      1.0
9      1.0
TOT    1.0
Name: PROP, dtype: float64

In [29]:
# calculate the weight_Group_inflation_2022_03 column based on 2022-03 and PROP
merged['weight_Group_inflation_2022_03'] = merged['2022-03'] * merged['PROP']

In [30]:
# calculate the weight_Group_inflation_2023_03 column based on 2023-03 and PROP
merged['weight_Group_inflation_2023_03'] = merged['2023-03'] * merged['PROP']

In [31]:
# calculate the weight_Group_inflation_2023_03 column based on 2023-03 and PROP
merged['weight_Group_inflation_Cumulated'] = merged['Cumulated'] * merged['PROP']

In [35]:
sumInflationMarch2022 = merged.groupby('DECUC')['weight_Group_inflation_2022_03'].sum()

In [36]:
sumInflationMarch2023 = merged.groupby('DECUC')['weight_Group_inflation_2023_03'].sum()

In [37]:
sumCumulated = merged.groupby('DECUC')['weight_Group_inflation_Cumulated'].sum()

In [38]:
sumCumulated[1] - sumCumulated[10]

0.10770954578776681